# (one-step) Biot equation

In this tutorial we present how to solve the (static) Biot equation with [PyGeoN](https://github.com/compgeo-mox/pygeon).  The unknown is the displacement $u$, the flux $q$ and the pressure $p$.

Let $\Omega$ with boundary $\partial \Omega$ and outward unit normal ${\nu}$. Given 
$\lambda$ Lamé constant and $\mu$ the Kirchhoff modulus, $K$ the permeability of the porous medium and $s_0$ its storativity, and $\alpha$ the Biot-Willis coefficient fot the coupling between the two models.
We want to solve the following problem: find $({u}, {q}, p)$ such that
$$
    \begin{aligned}
         & \nabla \cdot [2 \mu {\epsilon}({u}) + \lambda \nabla \cdot u I
         - \alpha p {I}] = -{b}   \\
         & \mu{q} + {K}\nabla p = {0}                        \\
         & \partial_t (s_0 p + \alpha \nabla \cdot {u}) +
        \nabla \cdot {q} = \psi
    \end{aligned}
    \quad \text{in } \Omega.
$$
with $\epsilon$ the symmetric gradient and $b$ a body force. The stress tensor, which can be post-processed from $u$, is given by
$$
    \sigma = 2 \mu \epsilon(u) + \lambda \nabla \cdot u I - \alpha p I
$$

In particular, we consider only the spatial structure of the previous problem by doing only one time step. We call the latter $\Delta t$ and write the discrete problem as
$$
    \begin{bmatrix}
        K & 0 & -\alpha D^\top \\
        0 & \Delta t M_q & - \Delta t B^\top \\
        \alpha D & \Delta t B & s_0 M_p
    \end{bmatrix}
    \begin{bmatrix}
        u \\ q \\ p
    \end{bmatrix}
    =
    \begin{bmatrix}
        b \\ 0 \\ \psi
    \end{bmatrix}
$$
where $K$ is the stiffness matrix associated with the elatic problem, $D$ is the coupling between the two physics, $M_q$ is the mass matrix associated to the flux variable, $B$ is the divergence matrix for the flow problem, and $M_p$ is the mass matrix associated with the pressure. All the aforementioned matrices are properly scaled by their physical parameters if not explicitly written. The second row has been multiply by $\Delta t$ to preserve the skew-symmetry of the problem.

## Exercise 1: fixed-strain iterative solver

To solve the discrete problem we employ the fixed-strain iterative solver which reads:
given $u^i$ compute $(u^{i+1}, q^{i+1}, p^{i+1})$ by solving until convergence the following steps:

$$
 \begin{bmatrix}
         \Delta t M_q & - \Delta t B^\top \\
         \Delta t B & s_0 M_p
    \end{bmatrix}
    \begin{bmatrix}
         q^{i+1} \\ p^{i+1}
    \end{bmatrix}
    =
    \begin{bmatrix}
       0 \\ \psi- D\alpha u^{i}
    \end{bmatrix}
$$

followed by
$$
K u^{i+1} = b + \alpha D^\top p^{i+1}
$$

Note that i) we are performing only one time step starting from $u=0$, $p=0$, therefore the right hand side is simplified and ii) the two system are equivalent to the following block triangular one:

$$
    \begin{bmatrix}
        K & 0 & -\alpha D^\top \\
        0 & \Delta t M_q & - \Delta t B^\top \\
        0 & \Delta t B & s_0 M_p
    \end{bmatrix}
    \begin{bmatrix}
        u^{i+1} \\ q^{i+1} \\ p^{i+1}
    \end{bmatrix}
    =
    -
    \begin{bmatrix}
        0 & 0 & 0 \\
        0 & 0 & 0 \\
        \alpha D& 0 & 0
    \end{bmatrix}
    \begin{bmatrix}
        u^{i} \\ q^{i} \\ p^{i}
    \end{bmatrix}
    +
    \begin{bmatrix}
        b \\ 0 \\ \psi
    \end{bmatrix}
$$

A footing problem is when a force is impose on the top compressing the body and the bottom is fixed. In this case we assume that the force is applied on the central portion of the top boundary.

For this test case we set $\Omega = [0, 3] \times [0, 1]$, $b = 0$, and the following boundary conditions:
$$ 
\begin{aligned}
    &u = 0 &&\text{ and }&& \nu \cdot q = 0 &&\text{ on } \partial \Omega  \setminus \partial_{top} \Omega
\\
&\nu \cdot \sigma = 0 &&\text{ and }&& p = 0 &&\text{ on } \partial_{top} \Omega \setminus \partial_s \Omega
\\
&\nu \cdot \sigma = [0, -1]^\top &&\text{ and }&& p = 0 &&\text{ on }\partial_s \Omega 
\end{aligned}
$$
where $\partial_{s} \Omega = [1, 2] \times \{1\}$.

We present *step-by-step* how to create the grid, declare the problem data, and finally solve the problem.

First we import some of the standard modules, like `numpy` and `scipy.sparse`. Since PyGeoN is based on [PorePy](https://github.com/pmgbergen/porepy) we import both modules.

In [1]:
import numpy as np
import scipy.sparse as sps

import porepy as pp
import pygeon as pg


# useful for the stopping criteria in the iterative solvers
def compute_err(x, x_i, mass):
    diff = np.sqrt((x_i - x) @ mass @ (x_i - x))
    norm = np.sqrt(x_i @ mass @ x_i)
    return diff / norm if norm else diff

We create now the grid, since we use a vector Lagrangian of order 1 for ${u}$ and Raviart-Thomas for the flux $q$, we are restricted to simplices. In this example we consider a 2-dimensional structured grid, but the presented code will work also in 3d.

In [2]:
mesh_size = 0.05
delta_t = 0.01
dim = 2

nx = np.array([30, 10])
sd = pp.StructuredTriangleGrid(nx, [1] * dim)
sd.nodes[0, :] *= 3

pg.convert_from_pp(sd)
sd.compute_geometry()

# data for the iterative solvers
tol = 1e-4
max_iter = 1e2

Let us declare the finite element spaces that we are going to use

In [3]:
key = "biot"

# definition of the discretizations
vec_p1 = pg.VecLagrange1(key)
rt0 = pg.RT0(key)
p0 = pg.PwConstants(key)

# dofs for each variable
dofs = np.array([vec_p1.ndof(sd), rt0.ndof(sd), p0.ndof(sd)])

With the following code we set the data and the boundary conditions. Since we need to identify each side of $\partial \Omega$ we need few steps.

In [4]:
# the physical parameters of the problem, assumed constant
lambda_ = 1
mu = 0.5
alpha = 1
s0 = 1  # 1 0.5

inv_perm = pp.SecondOrderTensor(np.ones(sd.num_cells))
param = {pg.SECOND_ORDER_TENSOR: inv_perm, pg.LAME_LAMBDA: lambda_, pg.LAME_MU: mu}
data = pp.initialize_data({}, key, param)

# selection of the boundary conditions
bd_q = sd.tags["domain_boundary_faces"]
bd_q[np.isclose(sd.face_centers[1, :], 1)] = False

bd_u = sd.tags["domain_boundary_nodes"]
bd_u[np.isclose(sd.nodes[1, :], 1)] = False
bd_u = np.hstack([bd_u] * dim)

top_s = np.logical_and.reduce(
    (
        np.isclose(sd.face_centers[1, :], 1),
        sd.face_centers[0, :] > 1,
        sd.face_centers[0, :] < 2,
    )
)

fun = lambda _: np.array([0, -1])

Once the data are assigned to the grid, we construct the matrices. Once the latter is created, we also construct the right-hand side containing the boundary conditions.

In [5]:
# construction of the block matrices
mass_u = vec_p1.assemble_mass_matrix(sd)
mass_q = rt0.assemble_mass_matrix(sd, data)
mass_p = p0.assemble_mass_matrix(sd)

div_q = mass_p @ rt0.assemble_diff_matrix(sd)
div_u = mass_p @ vec_p1.assemble_div_matrix(sd)

stiff_u = vec_p1.assemble_stiff_matrix(sd, data)

We need to solve the linear system, PyGeoN provides a framework for that. The actual imposition of essential boundary conditions (displacement boundary conditions) might change the symmetry of the global system, the class `pg.LinearSystem` preserves this structure by internally eliminating these degrees of freedom.

In [6]:
# construction of the matrices for the sub_problems and the RHS (the fixed part)
sp_fluid = sps.block_array(
    [
        [delta_t * mass_q, -delta_t * div_q.T],
        [delta_t * div_q, s0 * mass_p],
    ],
    format="csc",
)

sp_mech = stiff_u

bd_mech = vec_p1.assemble_nat_bc(sd, fun, top_s)
bd_fluid = np.hstack((bd_q, np.zeros(dofs[2], dtype=bool)))

step = 0
err = tol + 1

# initialization of the solution
u_i = np.zeros(dofs[0])
q_i = np.zeros(dofs[1])
p_i = np.zeros(dofs[2])

while err > tol and step < max_iter:
    # for a given u solve the flow problem
    rhs_fluid = np.zeros(dofs[1:].sum())
    rhs_fluid[-dofs[2] :] = -alpha * div_u @ u_i

    ls1 = pg.LinearSystem(sp_fluid, rhs_fluid)
    ls1.flag_ess_bc(bd_fluid, np.zeros_like(rhs_fluid))
    x = ls1.solve()

    # split of the solution from the vector x
    idx = np.cumsum(dofs[1:-1])
    q, p = np.split(x, idx)

    # for a given p solve the mech problem
    rhs_mech = bd_mech + alpha * div_u.T @ p

    ls2 = pg.LinearSystem(sp_mech, rhs_mech)
    ls2.flag_ess_bc(bd_u, np.zeros_like(rhs_mech))
    u = ls2.solve()

    # compute the stopping criteria
    step += 1
    err_u = compute_err(u_i, u, mass_u)
    err_q = compute_err(q_i, q, mass_q)
    err_p = compute_err(p_i, p, mass_p)

    u_i = u.copy()
    q_i = q.copy()
    p_i = p.copy()

    err = err_u + err_q + err_p

    print(step, err, err_u, err_q, err_p)

1 1.0 1.0 0.0 0.0
2 2.479112492542064 0.4791124925420638 1.0 1.0
3 1.7998595102351698 0.20061011364639664 0.6896122192324553 0.9096371773563179
4 0.6867007412857906 0.10584073733520963 0.2654094794479815 0.31545052450259947
5 0.3882107339069305 0.050921726229185525 0.15126467016738104 0.18602433751036396
6 0.18626249182091575 0.02596714391515255 0.07366454909058152 0.08663079881518168
7 0.09793009050430276 0.012976460460772025 0.03914553809081089 0.04580809195271986
8 0.04934937148352772 0.006589383044699428 0.01992581732043115 0.02283417111839714
9 0.02543947806466951 0.0033331250510880168 0.01035114175301649 0.011755211260565003
10 0.01296729011637011 0.0016943828684740435 0.005310166929034778 0.00596274031886129
11 0.006647086954145976 0.0008610790959426484 0.0027358109382026393 0.003050196920000688
12 0.0033967496848992754 0.00043838707997185936 0.0014037680296000092 0.001554594575327407
13 0.001738059310949813 0.0002232646452582013 0.0007206479806656249 0.0007941466850259868
14 0.

We post-process now the stress tensor, one `sd.dim` x `sd.dim` tensor per cell.

In [7]:
cell_sigma = vec_p1.compute_stress(sd, u, data)

# split the tensor in each component for the exporting
cell_sigma_xx = cell_sigma[:, 0, 0]
cell_sigma_xy = cell_sigma[:, 0, 1]
cell_sigma_yy = cell_sigma[:, 1, 1]

Since the computed $u$ is a vector per peak of the grid, for visualization purposes we project the displacement in each cell center as vector. Similarly for the flow field $q$. We finally export the solution to be visualized by [ParaView](https://www.paraview.org/).

In [8]:
# compute the cell flow, one vector per cell
proj_q = rt0.eval_at_cell_centers(sd)
cell_q = (proj_q @ q).reshape((3, -1))
cell_p = p0.eval_at_cell_centers(sd) @ p
u_3d = np.hstack((u, np.zeros(sd.num_nodes))).reshape((3, -1))

# export the final solution
save = pp.Exporter(sd, "sol", folder_name="ex1")
save.write_vtu(
    [
        ("cell_q", cell_q),
        ("cell_p", cell_p),
        ("cell_sigma_xx", cell_sigma_xx),
        ("cell_sigma_xy", cell_sigma_xy),
        ("cell_sigma_yy", cell_sigma_yy),
    ],
    data_pt=[("u", u_3d)],
)

In [9]:
# compute the spectral radius of the iteration matrix
# preconditioner P
# fmt: off
P = sps.block_array(
    [
        [stiff_u,             None,   -alpha * div_u.T],
        [   None, delta_t * mass_q, -delta_t * div_q.T],
        [   None,  delta_t * div_q,        s0 * mass_p],
    ]
)

# P-A
M = sps.block_array(
    [
        [   0 * stiff_u,       None,       None],
        [          None, 0 * mass_q,       None],
        [-alpha * div_u,       None, 0 * mass_p],
    ]
)
# fmt: on
# compute the 6 largest magnitude eigs for the generalized problem M x = lambda P x
ll = sps.linalg.eigs(M, 6, P, None, "LM")
print(ll)

(array([-0.57243501+0.j, -0.54570074+0.j, -0.50687641+0.j, -0.46168234+0.j,
       -0.42784687+0.j, -0.41503016+0.j]), array([[ 3.01423189e-01+0.j, -1.23564742e-01+0.j, -1.34665422e-01+0.j,
         9.26238661e-02+0.j, -5.90179919e-01+0.j, -1.03596066e-01+0.j],
       [ 3.00865661e-01+0.j, -1.22208008e-01+0.j, -1.32298822e-01+0.j,
         8.93287147e-02+0.j, -5.88986473e-01+0.j, -1.07108966e-01+0.j],
       [ 2.98135136e-01+0.j, -1.16119394e-01+0.j, -1.22822947e-01+0.j,
         7.77355378e-02+0.j, -5.84370121e-01+0.j, -1.17581215e-01+0.j],
       ...,
       [-1.85427994e-04+0.j, -3.74679963e-04+0.j,  5.36979978e-04+0.j,
         6.27540779e-04+0.j, -3.19304305e-04+0.j, -5.38483987e-04+0.j],
       [-8.39468268e-05+0.j, -1.74722166e-04+0.j,  2.62305254e-04+0.j,
         3.27305206e-04+0.j, -1.42590615e-04+0.j, -3.18053347e-04+0.j],
       [-8.48937129e-05+0.j, -1.75780198e-04+0.j,  2.61542247e-04+0.j,
         3.21725189e-04+0.j, -1.50805287e-04+0.j, -3.02064518e-04+0.j]],
      shap

In [10]:
# Consistency check
assert np.isclose(np.linalg.norm(cell_q), 10.567237889498022)
assert np.isclose(np.linalg.norm(cell_p), 3.1286666278129225)
assert np.isclose(np.linalg.norm(cell_sigma), 10.026256065440071)
assert np.isclose(np.linalg.norm(u), 2.2684117671909236)